## Expected Improvement calcualtion for large number of candidate space (74 million)

#### The number of candidates are more than 74 million. Therfore to avoid any memory error, chunk wise data reading and execution has been done. We are considering 1 million as chunk size. At present we are executing sequentially however parallel computing can be done  
### ** Better works in VS Code Editor

In [ ]:
from itertools import count
from operator import ge
import pandas as pd 
import numpy as np
from cbfv.composition import generate_features
from convesrion_weightpct_atpct import wtpct_To_atpct
import matplotlib.pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import math
from tqdm import tqdm
rng_seed = 20



In [ ]:

def generate_atpct(in_path, out_path):
    '''
    Convert weigth percent to atom percent 
    
    Paramenters:
    Input file path name consisting of formula in wt percent, Temperature and Target
    Outful file path name where the atom percent data will save after conversion 
    '''
    with open(in_path) as wtpct:
        with open(out_path, 'w') as atpct:
            atpct.write('formula,T,target\n')
            next(wtpct)
            for l in wtpct:
                l_str = l.split(",")
                formula = wtpct_To_atpct(l_str[0])
                atpct.write(f'{formula},{l_str[1]},{l_str[2]}')


def generate_CBFV_features(in_path,out_path,nlines):
    
    '''
    Featurization using CBFV 
    
    Paramenters:
    Input file path name consisting of formula in atom percent, Temperature and Target
    Outful file path name where the featurization vector will save
    '''
    
    with open(in_path,'r') as input_file:
        length = len(input_file.readlines())

    ### number of loops = length/nlines
    no_loops = math.ceil(length/nlines)

    with open(out_path, 'a') as features:

        features.truncate(0)
        header = ["formula", "T", "target"]
        
        
        for i in np.arange(no_loops):
            df = pd.read_csv(in_path,nrows=nlines,skiprows=i*nlines+1, header=None)
            df.columns=header
            
            X_test, y_test, formulae_test, skipped_train  = generate_features(df,elem_prop='f3_revised',drop_duplicates=False,extend_features=True)
            
            X_test_avg = X_test[['avg_Atomic_Radius','avg_Pauling_Electronegativity','avg_number_of_valence_electrons','avg_Cohesive_energy_ev_atom',
            'avg_Bulk_modulus_RT_Gpa','avg_Elastic_modulus_RT_Gpa','avg_Shear_modulus_RT_Gpa','avg_Melting_point_(K)','avg_rate_shear_mod_Mpa_perK',
            'avg_Solid_Solubility_atpct','avg_lattice_constant_A','avg_BEC_percm3','avg_Av.Valence_bond_strength_ev','avg_EngelZ_e/a','T']]

            df_final= pd.concat([formulae_test,X_test_avg],axis=1)

            if i == 0:
                features.write(df_final.to_csv(index=False))
            else:
                features.write(df_final.to_csv(index=False, header=None))   
                


def model_fitting_bootstrap(X_train,y_train,n_iter=1000, seed = 20):
    
    '''
    Fitting the machine learning model with X_train and y_train data by using bootstrapping
    
    paramenters:
    X_train = training features
    y_tain = training target
    n_iter = no of bootstrap sample
    seed = 20 is fixed to have repeatability in boot straping
    
    return:
    returns trained models depending on how many bootstrap sample have been choosen 
    '''
    
   #UTS 
    parameters = dict(criterion='mse', max_features='auto',
                          min_samples_leaf=6, min_samples_split=5,
                          n_estimators=50, random_state=20)
    
#     
#   # YS
#     parameters = dict(criterion='mse', max_features='auto',
#                           min_samples_leaf=4, min_samples_split=5,
#                           n_estimators=200, random_state=20)

    # trained model list to hold all 1000 trained model
    trained_models = []
    
    # index required for bootstrapping 
    index = np.arange(X_train.shape[0]) 
    
    # setting the random seed to ensure same result 
    np.random.seed(seed)
    
    for i in range(n_iter):

        #sample from X_train, y_train with replacement
        index_sampled = np.random.choice(index, size=X_train.shape[0], replace=True)
        
        #print(index_sampled)

        X_train_sample = X_train[index_sampled,:]
        y_train_sample = y_train[index_sampled]
        
        
        # Instantiate the GBR model 
        gbr = GradientBoostingRegressor(**parameters)
        gbr.fit(X_train_sample, y_train_sample)
        
        trained_models.append(gbr)
        
    return trained_models



def model_predicting(trained_model,X_test):
    '''
    creating a 2d array to store prediction from all the models available in trained_model
    parmeters:
    trained_models and X_test 
    return:
    mean and std for each test data 
    '''
    
    #shape :X_test length as no of rows and total no of models as column 
    bootstrap_preds = np.zeros([len(X_test), len(trained_model)])  
    
    
    for i,model in enumerate(trained_model):
        pred_i = model.predict(X_test)
        #print(pred_i)
        bootstrap_preds[:,i] = pred_i
        
    return(bootstrap_preds.mean(1),bootstrap_preds.std(1))




def Expected_Improvement(mu_candidate, sigma_candidate, y_train, xi=0.01):

    '''
     Calcualting the Expected Improvement
     More information : http://krasserm.github.io/2018/03/21/bayesian-optimization/
     parameters: mean, std, y_train, xi
     return :
     Expected improvement value
     
    '''
     
    
    mu_max = np.max(y_train) 
    # mu_max is the highest material property value in the training data which is at 24 deg C **** Important
    # therefore the candidate search space is choosen at 24 deg C
    
    diff = (mu_candidate - mu_max - xi)
    z = diff/sigma_candidate
    ei = diff*norm.cdf(z)+sigma_candidate * norm.pdf(z)
    ei[sigma_candidate == 0.0] = 0.
    
    return(ei, mu_candidate, sigma_candidate)



def generate_ei(train_data_path, candidates_data_path, ei_outpath, nlines = 1000000, seed = 20):
    
    
    '''
     Generate the file consisting of Expected Improvement value for each candidate alloy 
     parameters: 
     train data path, candidate data path, ei outpath, no of lines to read at chunk 
     return :
     Expected improvement value
     
    '''
    
    
    ## reading csv having entire training data 
    df_train = pd.read_csv(train_data_path)
    X_train_unscale, y_train, formulae_train, skipped_train  = generate_features(
                                                                    df_train,
                                                                    elem_prop='f3_revised',
                                                                    drop_duplicates=False,
                                                                    extend_features=True)

    # considering only the average and temperature
    # TODO : use regex for getting the average clumn names from X_train.columns
    X_train_avg = X_train_unscale[['avg_Atomic_Radius','avg_Pauling_Electronegativity','avg_number_of_valence_electrons','avg_Cohesive_energy_ev_atom',
                                   'avg_Bulk_modulus_RT_Gpa','avg_Elastic_modulus_RT_Gpa','avg_Shear_modulus_RT_Gpa','avg_Melting_point_(K)','avg_rate_shear_mod_Mpa_perK',
                                   'avg_Solid_Solubility_atpct','avg_lattice_constant_A','avg_BEC_percm3','avg_Av.Valence_bond_strength_ev','avg_EngelZ_e/a','T']]
    
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_avg)

    
    trained_models = model_fitting_bootstrap(X_train,y_train,n_iter=1000, seed = seed)
    
    
    with open(candidates_data_path,'r') as input_file:
        candiate_length = len(input_file.readlines())

    ### number of loops = length/nlines
    no_loops = math.ceil(candiate_length/nlines)
    
    print("Total candidates ",candiate_length,flush=True)
    print("Number of loops ",no_loops, flush=True)
    with open(ei_outpath, 'a') as ei_file:
        ei_file.truncate(0)

        #TODO: use tqdm to check the progress
        for i in tqdm(np.arange(no_loops)):
            df = pd.read_csv(candidates_data_path,nrows=nlines,skiprows=i*nlines+1, header=None)
            formula = df.iloc[:,0]
            df.drop(columns=df.columns[0],axis=1,inplace=True)#drop formula column
            df.columns=np.arange(15)#TODO: remove this redundant line
            
            X_test = scaler.transform(df)
            mean, std = model_predicting(trained_models, X_test)
            ei, _, _ = Expected_Improvement(mean, std, y_train)


            df_out = pd.DataFrame({'formula':formula, 'mean': mean, 'std':std, 'ei':ei})
            
            if i == 0:
                ei_file.write(df_out.to_csv(index=False))
                
            else:
                #features.write('\n')
                ei_file.write(df_out.to_csv(index=False, header=None))







def sorting_EI(ei_file_path,out_path,nlines = 1000000):
    
    # It reads the number of lines in chunk and sort the first 20 alloy compositions from each chunk based on ei value and save them in a .csv file
    # It can be regarded as interim sorting 
    '''
    
     The function takes the 'ei' file as input 
     Sorting EI and storing 
     parameters: ei_file_path, out_path, nlines
     return :
     Sorted Expected improvement value from each chunk
     
    '''

    with open(ei_file_path,'r') as input_file:
        length = len(input_file.readlines())

    ### number of loops = length/nlines
    no_loops = math.ceil(length/nlines)

    with open(out_path, 'a') as ei_sort:
        ei_sort.truncate(0)

        #TODO: use tqdm to check the progress
        for i in tqdm(np.arange(no_loops)):
            df_ei = pd.read_csv(ei_file_path,nrows=nlines,skiprows=i*nlines+1, header=None)
            df_ei.columns= ['formula','mean','std','ei']
            df_ei_final = df_ei.sort_values('ei',ascending=False)[0:20]
            if i == 0:
                ei_sort.write(df_ei_final.to_csv(index=False))
            else:
                #features.write('\n')
                ei_sort.write(df_ei_final.to_csv(index=False, header=None))




def sorting_mean(ei_file_path,out_path,nlines = 1000000):
    
    
    # It reads the number of lines in chunk and sort the first 20 alloy compositions from each chunk based on mean value and save them in a .csv file
    # It can be regarded as interim sorting 
    '''
    
     The function takes the 'ei' file as input 
     Sorting mean and storing 
     parameters: ei_file_path, out_path, nlines
     return :
     Sorted mean value from each chunk
     
    '''


    with open(ei_file_path,'r') as input_file:
        length = len(input_file.readlines())

    ### number of loops = length/nlines
    no_loops = math.ceil(length/nlines)

    with open(out_path, 'a') as mean_sort:
        mean_sort.truncate(0)

        #TODO: use tqdm or someting to check the progress
        for i in tqdm(np.arange(no_loops)):
            df_mean = pd.read_csv(ei_file_path,nrows=nlines,skiprows=i*nlines+1, header=None)
            df_mean.columns= ['formula','mean','std','ei']
            df_mean_final = df_mean.sort_values('mean',ascending=False)[0:20]
            if i == 0:
                mean_sort.write(df_mean_final.to_csv(index=False))
            else:
                #features.write('\n')
                mean_sort.write(df_mean_final.to_csv(index=False, header=None))


### Read the candidate alloy data and convert weight percent to atom percent 

In [ ]:
in_path = "data/compositions_test_24degc_allcombinations_5elements.csv"
atpct_path = "data/quinary/compositions_test_24degc_allcombinations_atpct_5.csv"
generate_atpct(in_path, atpct_path)



### Generate features using the alloy composition in atom percent

In [ ]:
features_out_path = "data/quinary/compositions_test_24degc_allcombinations_features_5.csv"
generate_CBFV_features(atpct_path, features_out_path, 1000000)



### Calculate EI for each candidate 


In [ ]:
features_out_path = "data/quinary/compositions_test_24degc_allcombinations_features_5.csv"
train_data_path = 'data/model_input/train_ultimatestrength_all.csv'
candidates_data_path = features_out_path
ei_outpath = 'data/quinary/ei_UTS_5.csv'

generate_ei(train_data_path,candidates_data_path,ei_outpath,nlines = 1000000)




In [ ]:
## sorting EI_interim
out_path = 'data/quinary/ei_UTS_sort_intm_5'
sorting_EI(ei_file_path=ei_outpath, out_path=out_path, nlines=1000000)



### Saving in .csv file based on suggested candidates 

In [ ]:
df_ei = pd.read_csv(out_path)
df_ei_sorted = df_ei.sort_values('ei',ascending=False)
df_ei_sorted.to_csv('data/quinary/UTS_ei_sort_final_5.csv')

